https://tushare.pro/document

In [1]:
# %pip install tushare --upgrade
# %pip install pyyaml
# %pip install matplotlib
# %pip install seaborn

In [2]:
_PATH = '/Users/winston/git_rep/sys23/data/'

In [3]:
import os
import pandas as pd
import numpy as np
import datetime
import time
import importlib
from tqdm import tqdm
from matplotlib import pyplot as plt

In [4]:
import yaml
conf = '/Users/winston/git_rep/sys23/config_stk.yaml'
conf = yaml.safe_load(open(conf, encoding='utf-8'))

import sys
sys.path.append(conf['api_path'])
# from plt_head import *
import data.api
import importlib
importlib.reload(data.api)

import tushare as ts
# _TOKEN = open(conf['tushare_token']).read()
# ts.set_token(_TOKEN)
# pro = ts.pro_api(_TOKEN)
pro = ts.pro_api()

## 0203 获取个股；个股后复权盘面可有日度记录确定

### 交易日期

In [5]:
start_date = '20150101'
end_date = '20230130'

tradedates = data.api.get_tradedates(start_date, end_date, pro=pro, fmt='%Y-%m-%d')

get_tradedates: 2015-01-05 2023-01-30


### IPO新股列表 最大2000条

In [6]:
new_share = data.api.wget_new_share(pro, path=_PATH + 'cache/', 
                                    lfy=2008, rfy=2023, step=5)
new_share.head()

Load 3684 rows from `/Users/winston/git_rep/sys23/data/cache/new_share.csv`
Save 3693 rows in `/Users/winston/git_rep/sys23/data/cache/new_share.csv`


,ts_code,sub_code,name,ipo_date,issue_date,amount,market_amount,price,pe,limit_amount,funds,ballot
0,001278.SZ,001278,一彬科技,20230221,,3093.0,0.0,0.00,0.0,1.2,5.259,0.0
1,688522.SH,787522,纳睿雷达,20230220,,3867.0,0.0,0.00,0.0,1.1,10.667,0.0
2,301408.SZ,301408,华人健康,20230220,,6001.0,0.0,0.00,0.0,1.1,0.000,0.0
3,603065.SH,732065,宿迁联盛,20230217,,4190.0,0.0,0.00,0.0,1.2,5.384,0.0
4,834770.BJ,889883,艾能聚,20230215,,2000.0,1600.0,5.98,16.0,95.0,1.196,0.0


### 指数成分和权重

In [7]:
for kind in ['CSI300', 'CSI500', 'CSI800', 'CSI1000']:
    
    index_code = conf['csi_pool'][kind]; print(kind, index_code)
    
    df = data.api.wget_index_weight(pro, index_code, 2014, 2024,
                                    path=_PATH + 'cache/')
df.head()

CSI300 399300.SZ
399300.SZ FY2023 Q2: no index_weight records
399300.SZ FY2023 Q3: no index_weight records
399300.SZ FY2023 Q4: no index_weight records
CSI500 000905.SH
000905.SH FY2023 Q2: no index_weight records
000905.SH FY2023 Q3: no index_weight records
000905.SH FY2023 Q4: no index_weight records
CSI800 000906.SH
000906.SH FY2023 Q2: no index_weight records
000906.SH FY2023 Q3: no index_weight records
000906.SH FY2023 Q4: no index_weight records
CSI1000 000852.SH
000852.SH FY2023 Q2: no index_weight records
000852.SH FY2023 Q3: no index_weight records
000852.SH FY2023 Q4: no index_weight records


,index_code,con_code,trade_date,weight
0,000852.SH,600680.SH,20141231,0.04
1,000852.SH,001872.SZ,20141231,0.11
2,000852.SH,001914.SZ,20141231,0.11
3,000852.SH,600680.SH,20141128,0.05
4,000852.SH,001872.SZ,20141128,0.08


In [8]:
# ## 个股复权历史
# df = ts.pro_bar(ts_code='600519.SH', adj='hfq', start_date=start_date, end_date=end_date)

# df = df.set_index(pd.to_datetime(df.trade_date))
# df.close.plot()